In [1]:
import pandas as pd
import numpy as np
import datetime
import pandas as pd

In this final, you will be asked to analyze a data set containing historical reservations for a fine-dining restaurant that is stored in the file "data/Reservation_Data.csv".  This restaurant only accepts reservations booked online and hence does not accept walk in customers.   Each row in this data corresponds to a booked reservation.  The columns have the following meaning:
- reservation\_date: The date that the reservation was booked for. In other words, this is the date when the customer will dine.
- reservation\_time: The time that the reservation was booked for.
- reservation\_party\_size: The size of the party for the corresponding reservation, i.e. the number of diners.
- reservation_date_booked: the date on which the reservation was made.
- datetime\_booked: The date and time corresponding to when the reservation was made (in UTC). This column has missing values, which have been entered as "#N/A".  **The restaurant is located in a Pacific time zone.**

Your goal in this final will be to undertand how customers schedule reservations at this restaurant.


# Part 1 

Your first task is to read in the data and do the following:

- delete rows with missing values
- convert datetime_booked to a datetime column with Pacific time zone
- combine reservation_date and reservation time to create a new column called reservation_datetime that is a datetime column that has a Pacific time zone. So the final dataframe you return should have 6 columns in total.
- Only keep reservations made at the following 8 time slots: 17:30, 17:45, 18:00, 18:15, 20:45, 21:00, 21:15, 21:30

Return this modified version of the original data frame.

In [2]:
def Read_Data():
    
    df_res = pd.read_csv("data/Reservation_Data.csv", na_values=["#N/A"])
    df_res = df_res.dropna(axis=0)
    df_res.datetime_booked=pd.to_datetime(df_res.datetime_booked)#-pd.Timedelta(hours=8)
    df_res.datetime_booked=df_res.datetime_booked.dt.tz_localize("GMT").dt.tz_convert\
    ("America/Los_Angeles").dt.tz_localize(None)
    df_res["reservation_datetime"]=pd.to_datetime(df_res["reservation_date"]+ " "+df_res["reservation_time"])


    time_slots=["17:30","17:45","18:00","18:15","20:45","21:00","21:15","21:30"]
    time_index=df_res.reservation_datetime.apply(lambda x: x.strftime("%H:%M") in time_slots)
    df_res_new=df_res.loc[time_index,]
    df_res_new
    return df_res_new


In [3]:
df_res = Read_Data()
assert df_res.shape==(3359,6)

In [4]:
assert np.isclose(df_res.datetime_booked.dt.hour.mean(), 13.55969)

# Part 2 

In this next part, we will write two functions to understand basic patterns in the data.

The first function takes as input one of the eight reservation time slots that we are considering as a datetime time and it outputs the day of the week (as a string) with the smallest average party size of all reservation made for the given inputted time slot.

In [5]:
def Get_Avg_Party_Size(res_time):
    
    smallest_dow = None
    df_res = Read_Data()
    df_res=df_res.loc[df_res.reservation_time==str(res_time)]
    df_res["day_of_week"]=pd.to_datetime(df_res.reservation_date).dt.day_name()
    smallest_dow=df_res.groupby(by="day_of_week").reservation_party_size.mean().idxmin() 
    
    return smallest_dow

In [6]:
res_time = datetime.time(hour = 17, minute=30)
assert Get_Avg_Party_Size(res_time)=="Sunday"

In [7]:
res_time = datetime.time(hour = 21, minute=0)
assert Get_Avg_Party_Size(res_time)=="Tuesday"

The second function I would like you to write takes as input a parameter called num_days (you may assume this is an integer), and it returns the fraction of reservations made by parties size 1/2 that are made at most num_days in advance, and then also the same fraction but just for parties of 3/4.

**NOTE: The number of days in advance that a reservation is made is (reservation_datetime - datetime_booked).**

In [8]:
def Booked_in_Advance(num_days):
    
    df_res = Read_Data()
    num_days = datetime.timedelta(days=num_days)
    party_one_two = None
    party_three_four = None


    num_days=num_days.days
    df_res["days_in_advance"]=df_res.apply(lambda x: (x["reservation_datetime"]-x["datetime_booked"]).days,axis=1)

    #dia<=num_days
    #1/2
    res_12_index=df_res.reservation_party_size.apply(lambda x: x==1 or x==2)
    res_12_all=df_res.loc[res_12_index,]
    res_12_required=res_12_all.loc[res_12_all.days_in_advance<=num_days-1]
    party_one_two=res_12_required.shape[0]/res_12_all.shape[0]


    #3/4
    res_34_index=df_res.reservation_party_size.apply(lambda x: x==3 or x==4)
    res_34_all=df_res.loc[res_34_index,]
    res_34_required=res_34_all.loc[res_34_all.days_in_advance<=num_days-1]
    party_three_four=res_34_required.shape[0]/res_34_all.shape[0]


    
    return party_one_two, party_three_four
    

In [9]:
result = Booked_in_Advance(7)
assert np.isclose(result[0],0.27188)

In [10]:
result = Booked_in_Advance(100)
assert np.isclose(result[1],0.860534)

# Part 3

Let's assume that there are two service periods:

- Service Period 1 consists of time slots 17:30, 17:45, 18:00, 18:15
- Service Period 2 consiss of time slots 20:45, 21:00, 21:15, 21:30

The next function takes as input res_time, which is one of the eight times above as a datetime time.  Given the inputted time, you will exclusively focus this analysis on either Service Period 1 or 2. For example, if the inputted res_time is 17:45, you should only consider reservations made during Service Period 1. For each day, find the first reservation that books at res_time (datetime_booked tells you the order in which reservations were scheduled).  Within the given Service Period of interest, compute how many reservations have already been booked for the given day before this reservation for res_time was made.  If $k-1$ reservations have already been made, this means that res_time was first booked as the $k$-th reservation.  We will say the "rank" of reservation time res_time on this day was $k$. So, if res_time was the first time booked on a particular day frot he relevant Service Period, its rank on this day is 1.  Return the average rank of res_time across all days in which a reservation for res_time was booked.


In [11]:
def Get_Avg_Rank(res_time):

    df_res = Read_Data()
    avg_Rank=None

    
    
    str_res_time=res_time.strftime("%H:%M")
    slot1=["17:30", "17:45", "18:00", "18:15"]
    slot2=["20:45", "21:00", "21:15", "21:30"]

    if str_res_time in slot1:
        slot=slot1[:]
    else:
        slot=slot2[:]

    df_res["str_rtime"]=df_res.reservation_time.apply(lambda x: str(x)[:-3])
    #在slot里的所有记录
    str_rtime_index=df_res.reservation_time.apply(lambda x: x in pd.to_datetime(slot).strftime("%T")) 
    df_res_in_slot=df_res.loc[str_rtime_index,]
    #res_time那个时间点的所有记录
    df_res_time=df_res.loc[df_res.str_rtime==str_res_time,]

    #每天的first reservation（根据datetime_booked）
    first_res_index=df_res_time.groupby(by="reservation_date").datetime_booked.idxmin()
    df_first_res=df_res_time.loc[first_res_index,]

    #每天first reservation被book前有多少
    ranks=[]
    for date in df_first_res.reservation_date:
        datetime_booked=df_first_res.loc[df_first_res.reservation_date==date,].datetime_booked.iloc[0]
    #    print(datetime_booked)
        df_date_index1=df_res_in_slot.datetime_booked.apply(lambda x: x<datetime_booked)
        df_new=df_res_in_slot.loc[df_date_index1,]
        df_date_index2=df_res_in_slot.reservation_date.apply(lambda x: x==date)
        df_new=df_new.loc[df_date_index2,]

        rank=len(df_new)+1
        ranks.append(rank)
    avg_rank=sum(ranks)/len(ranks)


    return avg_rank

In [12]:
res_time = datetime.time(hour = 21, minute=30)
assert np.isclose(Get_Avg_Rank(res_time),3.9035 )

In [13]:
res_time = datetime.time(hour = 20, minute=45)
assert np.isclose(Get_Avg_Rank(res_time),1.45714)